In [1]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import config

from turb_vae.train2d import VAETrainer
torch.set_grad_enabled(False)


model = VAETrainer.load_from_checkpoint("../checkpoints/vae.ckpt")
encoder = model.vae.encoder
data = config.TurbVaeConfig.test_dataset
N1 = 1000
dl = torch.utils.data.DataLoader(data, batch_size=N1, num_workers=15)

/home/hudson/code/spack/opt/spack/linux-ubuntu24.04-skylake_avx512/gcc-13.2.0/miniconda3-24.3.0-ree6kqamgcbjhoziwoelsymk7uzvnxju/envs/turb/lib/python3.11/site-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 56 worker processes in total. Our suggested max number of worker in current system is 20, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/hudson/code/spack/opt/spack/linux-ubuntu24.04-skylake_avx512/gcc-13.2.0/miniconda3-24.3.0-ree6kqamgcbjhoziwoelsymk7uzvnxju/envs/turb/lib/python3.11/site-packages/pytorch_lightning/utilities/par

In [2]:
batch = next(iter(dl))

In [3]:
N2 = 1000
n, L0 = batch
mu_logvar = encoder(n.to('cuda'))
mu = mu_logvar[:, :encoder.out_channels].view(-1, encoder.out_channels * 8 * 8)
logvar = mu_logvar[:, encoder.out_channels:].view(-1, encoder.out_channels * 8 * 8)

eps = torch.randn(N2, 1, 1).to("cuda")

z = mu + torch.exp(logvar / 2) * eps

# combine N1 and N2 dimensions
z = z.view(N1*N2, -1)
z.shape

torch.Size([1000000, 1024])

In [7]:
# mean stats
mu_est = z.mean(axis=0, keepdim=True)
mu_est.mean(), mu_est.std() / np.sqrt(N1*N2)

(tensor(-0.0132, device='cuda:0'), tensor(2.7540e-05, device='cuda:0'))

In [10]:
mu_est[:,:5].cpu()

tensor([[ 0.0047, -0.0056, -0.0086,  0.0556,  0.0121]])

In [5]:
# covariance stats
cov_est = torch.cov(z.T)
cov_est[:5, :5]

tensor([[1.1776, 0.4984, 0.2609, 0.1904, 0.2301],
        [0.4984, 1.2404, 0.5112, 0.2450, 0.2054],
        [0.2609, 0.5112, 1.0716, 0.3906, 0.2264],
        [0.1904, 0.2450, 0.3906, 0.9960, 0.3839],
        [0.2301, 0.2054, 0.2264, 0.3839, 0.9990]], device='cuda:0')